<a href="https://colab.research.google.com/github/acrossariver/my_gcola/blob/main/%E3%81%93%E3%82%93%E3%81%A0%E3%81%A6%EF%BC%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title data_processor.py
%%writefile data_processor.py
import pandas as pd
import re
from collections import Counter
from itertools import combinations
import io
from datetime import datetime

class DataProcessor:
    def __init__(self, data_str: str):
        self.data_str = data_str
        self.df = None
        self.rules = {}
        self.month_to_season = {
            1: '初春', 2: '初春',
            3: '仲春', 4: '仲春',
            5: '初夏', 6: '初夏',
            7: '仲夏', 8: '仲夏',
            9: '初秋', 10: '仲秋',
            11: '晩秋', 12: '初冬'
        }
        self.ingredient_nutrients_map = {}

    def load_and_process_data(self):
        """
        文字列データからDataFrameを読み込み、前処理と分析を実行する
        """
        try:
            self.df = pd.read_csv(io.StringIO(self.data_str))
            print("データ文字列からDataFrameを読み込みました。")
            self._preprocess_ingredients()
            self._preprocess_cooking_methods()
            self._preprocess_tastes()
            self._analyze_data()
            print("データ処理と分析が完了しました。")
        except Exception as e:
            print(f"データ処理中にエラーが発生しました: {e}")

    def get_rules(self) -> dict:
        return self.rules

    def get_nutrients_for_ingredient(self, ingredient_name: str) -> str:
        """
        食材名に対応する栄養価情報を取得する
        """
        return self.ingredient_nutrients_map.get(ingredient_name, '')

    def _preprocess_ingredients(self):
        def extract(text):
            if not isinstance(text, str): return {}
            ingredients = {}
            matches = re.findall(r'(\S+):\s*.*?【(.*?)】', text)
            for ingredient, nutrients in matches:
                ingredient_name = ingredient.strip()
                nutrients_list = [n.strip() for n in nutrients.split('、')]
                ingredients[ingredient_name] = nutrients_list
                self.ingredient_nutrients_map[ingredient_name] = f"【{nutrients}】"
            return ingredients

        if '主な食材（分量目安つき）' in self.df.columns:
            self.df['parsed_ingredients'] = self.df['主な食材（分量目安つき）'].apply(extract)
        else:
            print("警告: '主な食材（分量目安つき）'列が見つかりません。")

    def _preprocess_cooking_methods(self):
        def extract(text):
            if not isinstance(text, str): return 'その他'
            if '炒める' in text: return '炒める'
            if '煮る' in text: return '煮る'
            if '焼く' in text: return '焼く'
            if '茹でる' in text: return '茹でる'
            return 'その他'

        if '作り方' in self.df.columns:
            self.df['main_cooking_method'] = self.df['作り方'].apply(extract)
        else:
            print("警告: '作り方'列が見つかりません。")

    def _preprocess_tastes(self):
        def extract(name):
            if not isinstance(name, str): return 'その他'
            if '和風' in name or '豆腐' in name or '味噌' in name or 'だし' in name: return '和食'
            if 'カレー' in name or 'シチュー' in name or 'グラタン' in name or 'ソテー' in name: return '洋食'
            if '中華' in name or '麻婆' in name or '回鍋肉' in name: return '中華'
            if 'ガパオ' in name or 'タンドリー' in name or 'トムヤム' in name: return 'エスニック'
            return 'その他'

        if '料理名' in self.df.columns:
            self.df['旬'] = self.df['料理名'].apply(extract)
        else:
            print("警告: '料理名'列が見つかりません。")

    def _analyze_data(self):
        self.rules = {}
        if 'parsed_ingredients' in self.df.columns:
            all_ingredients = [ing for ingredients in self.df['parsed_ingredients'] if isinstance(ingredients, dict) for ing in ingredients.keys()]
            ingredient_counts = Counter(all_ingredients)
            self.rules['common_ingredients'] = [ing for ing, count in ingredient_counts.items() if count > 1]

            co_occurrence = Counter()
            for ingredients_dict in self.df['parsed_ingredients']:
                if not isinstance(ingredients_dict, dict): continue
                ingredients_list = list(ingredients_dict.keys())
                for pair in combinations(ingredients_list, 2):
                    co_occurrence[tuple(sorted(pair))] += 1
            self.rules['co_occurrence'] = {frozenset(pair): count for pair, count in co_occurrence.items() if count > 1}

        if 'main_cooking_method' in self.df.columns and 'parsed_ingredients' in self.df.columns:
            cooking_method_patterns = {}
            for method in self.df['main_cooking_method'].unique():
                ingredients_by_method = [ing for sublist in self.df[self.df['main_cooking_method'] == method]['parsed_ingredients'].apply(lambda d: list(d.keys()) if isinstance(d, dict) else []) for ing in sublist]
                cooking_method_patterns[method] = [ing for ing, count in Counter(ingredients_by_method).items() if count > 1]
            self.rules['cooking_method_patterns'] = cooking_method_patterns

        if '旬' in self.df.columns and 'parsed_ingredients' in self.df.columns:
            taste_patterns = {}
            for taste in self.df['旬'].unique():
                ingredients_by_taste = [ing for sublist in self.df[self.df['旬'] == taste]['parsed_ingredients'].apply(lambda d: list(d.keys()) if isinstance(d, dict) else []) for ing in sublist]
                taste_patterns[taste] = [ing for ing, count in Counter(ingredients_by_taste).items() if count > 1]
            self.rules['taste_patterns'] = taste_patterns

    def get_random_existing_menu(self) -> dict:
        if self.df is not None and not self.df.empty and '料理名' in self.df.columns and '主な食材（分量目安つき）' in self.df.columns and '作り方' in self.df.columns:
            random_row = self.df.sample(n=1).iloc[0]
            ingredients_str = random_row['主な食材（分量目安つき）']
            parsed_ingredients = {}
            matches = re.findall(r'(\S+):\s*.*?【(.*?)】', ingredients_str)
            for ingredient, nutrients in matches:
                parsed_ingredients[ingredient.strip()] = [n.strip() for n in nutrients.split('、')]

            return {
                'name': random_row['料理名'],
                'main_ingredients': list(parsed_ingredients.keys()),
                'main_ingredients_full': ingredients_str,
                'recipe': random_row['作り方'],
                'taste': random_row.get('旬', 'なし')
            }
        return None

    def get_seasonal_menu(self) -> dict:
        current_month = datetime.now().month
        season_name = self.month_to_season.get(current_month, 'なし')

        if '旬' in self.df.columns:
            seasonal_df = self.df[self.df['旬'].str.contains(season_name, na=False)]
        else:
            print("警告: '旬'列が見つからないため、'旬'列を生成して季節を判定します。")
            self._preprocess_tastes()
            seasonal_df = self.df[self.df['旬'].str.contains(season_name, na=False)]

        if not seasonal_df.empty:
            random_row = seasonal_df.sample(n=1).iloc[0]
            ingredients_str = random_row['主な食材（分量目安つき）']
            parsed_ingredients = {}
            matches = re.findall(r'(\S+):\s*.*?【(.*?)】', ingredients_str)
            for ingredient, nutrients in matches:
                parsed_ingredients[ingredient.strip()] = [n.strip() for n in nutrients.split('、')]

            return {
                'name': random_row['料理名'],
                'main_ingredients': list(parsed_ingredients.keys()),
                'main_ingredients_full': ingredients_str,
                'recipe': random_row['作り方'],
                'taste': random_row.get('旬', 'なし'),
                'source': '旬'
            }
        return None

    def get_herb_menu(self, herb: str) -> dict:
        if not self.df.empty and '主な食材（分量目安つき）' in self.df.columns:
            filtered_df = self.df[self.df['主な食材（分量目安つき）'].str.contains(herb, case=False, na=False)]
            if not filtered_df.empty:
                random_row = filtered_df.sample(n=1).iloc[0]
                ingredients_str = random_row['主な食材（分量目安つき）']
                parsed_ingredients = {}
                matches = re.findall(r'(\S+):\s*.*?【(.*?)】', ingredients_str)
                for ingredient, nutrients in matches:
                    parsed_ingredients[ingredient.strip()] = [n.strip() for n in nutrients.split('、')]

                return {
                    'name': random_row['料理名'],
                    'main_ingredients': list(parsed_ingredients.keys()),
                    'main_ingredients_full': ingredients_str,
                    'recipe': random_row['作り方'],
                    'taste': random_row.get('旬', 'なし'),
                    'source': 'ハーブ（既存）'
                }
        return None

Overwriting data_processor.py


In [ ]:
# @title menu_generator.py
%%writefile menu_generator.py
import random
from collections import Counter

class MenuGenerator:
    def __init__(self, rules: dict, data_processor):
        self.rules = rules
        self.data_processor = data_processor

    def generate_new_menu(self, ingredients_list: list) -> dict:
        """
        指定された食材リストから新しい献立を生成する
        Args:
            ingredients_list: 献立に使用する食材のリスト
        Returns:
            new_menu: 生成された献立のデータ
        """
        available_main_ingredients = [ing for ing in ingredients_list if ing in self.rules['cooking_method_patterns'].get('炒める', []) or ing in self.rules['cooking_method_patterns'].get('煮る', [])]
        if not available_main_ingredients:
            return None

        main_ingredient = random.choice(available_main_ingredients)

        side_dish_candidates = [ing for ing in ingredients_list if ing != main_ingredient]
        if not side_dish_candidates:
            return None

        side_dish = random.choice(side_dish_candidates)

        cooking_method = random.choice(list(self.rules['cooking_method_patterns'].keys()))
        menu_name = f"{main_ingredient}と{side_dish}の{cooking_method}物"

        main_ingredient_full = f"{main_ingredient}{self.data_processor.get_nutrients_for_ingredient(main_ingredient)}"
        side_dish_full = f"{side_dish}{self.data_processor.get_nutrients_for_ingredient(side_dish)}"
        ingredients_full_str = f"{main_ingredient_full}、{side_dish_full}"

        return {
            'name': menu_name,
            'main_ingredients': [main_ingredient, side_dish],
            'main_ingredients_full': ingredients_full_str,
            'cooking_method': cooking_method
        }

Overwriting menu_generator.py


In [ ]:
# @title recipe_creator.py
%%writefile recipe_creator.py
import re

class RecipeCreator:
    def __init__(self):
        self.templates = {
            '炒める': "1. {ingredient1}を炒めます。2. {ingredient2}を加えてさらに炒めます。3. 塩胡椒で味を調えます。",
            '煮る': "1. {ingredient1}と{ingredient2}を鍋に入れ、煮込みます。2. 蓋をして弱火で煮ます。",
            '焼く': "1. {ingredient1}と{ingredient2}を焼きます。2. 焼き目がついたら裏返します。",
            '茹でる': "1. {ingredient1}を茹でます。2. {ingredient2}を加えてさらに茹でます。"
        }

    def generate_simple_recipe(self, menu_data: dict) -> str:
        ingredients = menu_data['main_ingredients']
        method = menu_data['cooking_method']

        template = self.templates.get(method, "作り方のテンプレートが見つかりません。")

        try:
            return template.format(ingredient1=ingredients[0], ingredient2=ingredients[1])
        except (IndexError, KeyError):
            return "テンプレートの埋め込みに失敗しました。"

    def check_validity(self, recipe_text: str) -> bool:
        if "水を炒める" in recipe_text or "塩胡椒で味を調えます。1." in recipe_text:
            return False
        return True

    def rewrite_with_nlg(self, menu_data: dict) -> str:
        print("NLGモデルを呼び出し、作り方を修正しています...")
        ingredients_str = "、".join(menu_data['main_ingredients'])
        return f"【NLGによる修正】{menu_data['name']}の作り方:\n1. {ingredients_str}を{menu_data['cooking_method']}ます。2. 好みで味付けをしてください。"

Overwriting recipe_creator.py


In [ ]:
# @title gemini_recipe_generator.py
%%writefile gemini_recipe_generator.py
import google.generativeai as genai
import os

class GeminiRecipeGenerator:
    def __init__(self, api_key: str):
        if not api_key:
            raise ValueError("APIキーが設定されていません。")
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-2.5-flash-lite')

    def generate_herb_menu(self, ingredients: list, herb: str) -> dict:
        prompt = f"""
        以下の食材とハーブを使って、新しい料理の献立を日本語で提案してください。
        提案する料理の形式は以下の通りです。
        料理名: [料理名]
        主な食材: [食材1, 食材2, ...]
        作り方: [作り方の詳細]

        食材リスト: {", ".join(ingredients)}
        ハーブ: {herb}
        """
        print(f"Gemini APIにプロンプトを送信中: {prompt[:50]}...")
        try:
            response = self.model.generate_content(prompt)
            menu_data = self._parse_response(response.text)
            menu_data['source'] = 'AI生成'
            return menu_data
        except Exception as e:
            print(f"Gemini APIの呼び出し中にエラーが発生しました: {e}")
            return None

    def rewrite_recipe(self, menu_data: dict) -> str:
        prompt = f"""
        以下の料理の簡単な作り方を、より自然で分かりやすい日本語に書き直してください。

        料理名: {menu_data['name']}
        主な食材: {', '.join(menu_data['main_ingredients'])}
        元の作り方: {menu_data['recipe']}
        """
        print(f"Gemini APIにプロンプトを送信中: {prompt[:50]}...")
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"Gemini APIの呼び出し中にエラーが発生しました: {e}")
            return "レシピの書き換えに失敗しました。"

    def _parse_response(self, response_text: str) -> dict:
        menu_name = ""
        ingredients = []
        recipe = ""

        lines = response_text.split('\n')
        for line in lines:
            if line.startswith('料理名:'):
                menu_name = line.replace('料理名:', '').strip()
            elif line.startswith('主な食材:'):
                ingredients_str = line.replace('主な食材:', '').strip()
                ingredients = [i.strip() for i in ingredients_str.strip('[]').split(',')]
            elif line.startswith('作り方:'):
                recipe = line.replace('作り方:', '').strip()

        return {
            'name': menu_name,
            'main_ingredients': ingredients,
            'recipe': recipe
        }

Overwriting gemini_recipe_generator.py


In [ ]:
# @title main.py (現行版)
# -*- coding: utf-8 -*-
import os
import random
import sys
import pandas as pd
import re
import json
from collections import Counter
from itertools import combinations
from datetime import datetime, date
from IPython.display import display, HTML
from google.colab import drive

try:
    from google.colab import userdata
    from google.colab import auth
    from google.auth import default
except ImportError:
    print("Google Colab環境ではないようです。一部機能は動作しません。")

try:
    import google.generativeai as genai
except ImportError:
    print("Gemini APIライブラリがインストールされていません。AI生成機能は利用できません。")

def mount_drive():
    try:
        drive.mount('/content/drive')
        print("Google Driveをマウントしました。")
        return True
    except Exception as e:
        print(f"Google Driveのマウントに失敗しました: {e}")
        return False

def load_csv_from_drive(file_path: str):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        df.columns = df.columns.str.strip()
        print(f"'{file_path}' からCSVデータをDataFrameに読み込みました。")
        return df
    except FileNotFoundError:
        print(f"エラー: 指定されたファイル '{file_path}' が見つかりません。")
        return None
    except Exception as e:
        print(f"ファイルの読み込み中に予期せぬエラーが発生しました: {e}")
        return None

def load_main_keywords():
    """
    メイン食材のキーワードを固定値で返す。
    """
    #





    print("メイン食材キーワードを固定値で設定しました。")
    return ['鶏肉', '豚肉', '牛肉', '魚', '麺類', '鍋']

api_key = None
if 'google.colab' in sys.modules:
    try:
        api_key = userdata.get("GOOGLE_API_KEY")
        if not api_key:
            print("エラー: Colabのシークレット 'GOOGLE_API_KEY' が見つからないか、値が空です。")
    except Exception as e:
        print(f"Colabのシークレット取得中にエラーが発生しました: {e}")
else:
    api_key = os.getenv("GOOGLE_API_KEY")
    if not api_key:
        print("エラー: 環境変数 'GOOGLE_API_KEY' が設定されていません。")

def normalize_ingredient_name(name: str) -> str:
    normalization_map = {
        '玉ねぎ': 'タマネギ', 'たまねぎ': 'タマネギ', '人参': 'ニンジン', 'にんじん': 'ニンジン',
        'キャベツ': 'キャベツ', '鶏もも肉': '鶏肉', '豚ロース': '豚肉', '豚バラ': '豚肉',
        '鶏むね肉': '鶏肉', 'ホタルイカ': 'ホタルイカ', '豚挽き肉': '豚肉', '鶏挽き肉': '鶏肉', '合い挽き肉': '牛肉',
        '牛肉': '牛肉', '豚肉': '豚肉', '鶏肉': '鶏肉', '鮭': '鮭', '鯖': '鯖', 'カツオ': 'カツオ', 'アジ': 'アジ',
        '納豆': '納豆', '豆腐': '豆腐', '卵': '卵',
    }
    return normalization_map.get(name.strip(), name.strip())

def generate_menu_id(taste: str, sequence: int) -> str:
    today = datetime.now().strftime('%y%m%d')
    taste_map = {
        '和食': 'W',
        '洋食': 'Y',
        '中華': 'C',
        'エスニック': 'E',
        'ハーブ': 'H',
        '立春': 'S', '雨水': 'S', '啓蟄': 'S', '春分': 'S', '清明': 'S', '穀雨': 'S',
        '立夏': 'S', '小満': 'S', '芒種': 'S', '夏至': 'S', '小暑': 'S', '大暑': 'S',
        '立秋': 'S', '処暑': 'S', '白露': 'S', '秋分': 'S', '寒露': 'S', '霜降': 'S',
        '立冬': 'S', '小雪': 'S', '大雪': 'S', '冬至': 'S', '小寒': 'S', '大寒': 'S',
        'その他': 'O',
        '料理名指定': 'A',
        '特別ルール': 'S'
    }
    taste_abbr = taste_map.get(taste, 'O')
    return f"{today}-{taste_abbr}-{sequence:02d}"

class PantryTracker:
    def __init__(self, initial_pantry: list):
        self.pantry = Counter(normalize_ingredient_name(item) for item in initial_pantry)
        self.consumed_history = []
        self.user_pantry = Counter()
        self.initial_pantry_list = [normalize_ingredient_name(item) for item in initial_pantry]
        self.user_main_ingredients = []

    def consume_ingredients(self, ingredients: list):
        for ing in ingredients:
            normalized_ing = normalize_ingredient_name(ing)
            if self.pantry[normalized_ing] > 0:
                self.pantry[normalized_ing] -= 1
            if normalized_ing in self.user_main_ingredients:
                self.user_main_ingredients.remove(normalized_ing)
            self.consumed_history.append(normalized_ing)

    def get_available_ingredients(self):
        return [ing for ing, count in self.pantry.items() if count > 0]

    def add_ingredients(self, new_ingredients: list, is_main: bool = False):
        for ing in new_ingredients:
            normalized_ing = normalize_ingredient_name(ing)
            self.pantry[normalized_ing] += 1
            if is_main:
                self.user_main_ingredients.append(normalized_ing)
            self.user_pantry[normalized_ing] += 1

    def get_unused_user_main_ingredients(self):
        return [ing for ing in self.user_main_ingredients if self.pantry[ing] > 0]

    def get_user_ingredients(self):
        return [ing for ing, count in self.user_pantry.items() if count > 0]

    def get_initial_pantry_list(self):
        return self.initial_pantry_list

class DataProcessor:
    def __init__(self, df, main_keywords):
        self.df = df
        self.rules = {}
        self.solar_terms = {
            '立春': (2, 4, 2, 18), '雨水': (2, 19, 3, 4), '啓蟄': (3, 5, 3, 19),
            '春分': (3, 20, 4, 3), '清明': (4, 4, 4, 18), '穀雨': (4, 19, 5, 4),
            '立夏': (5, 5, 5, 19), '小満': (5, 20, 6, 4), '芒種': (6, 5, 6, 20),
            '夏至': (6, 21, 7, 6), '小暑': (7, 7, 7, 22), '大暑': (7, 23, 8, 7),
            '立秋': (8, 8, 8, 22), '処暑': (8, 23, 9, 7), '白露': (9, 8, 9, 22),
            '秋分': (9, 23, 10, 7), '寒露': (10, 8, 10, 23), '霜降': (10, 24, 11, 7),
            '立冬': (11, 8, 11, 21), '小雪': (11, 22, 12, 6), '大雪': (12, 7, 12, 21),
            '冬至': (12, 22, 1, 5), '小寒': (1, 6, 1, 19), '大寒': (1, 20, 2, 3)
        }
        self.ingredient_nutrients_map = {}
        self.main_keywords = main_keywords
        self.cooking_methods = ['炒める', '煮る', '焼く', '茹でる', '揚げる', '蒸す']

    @staticmethod
    def _parse_ingredients_string(text: str) -> dict:
        """Parses an ingredients string into a dictionary of ingredients and nutrients."""
        if not isinstance(text, str): return {}
        ingredients = {}
        matches = re.findall(r'【(\S+):\s*.*?】<(.*?)>', text)
        for ingredient, nutrients in matches:
            normalized_ingredient = normalize_ingredient_name(ingredient)
            nutrients_list = [n.strip() for n in nutrients.split('、')]
            ingredients[normalized_ingredient] = nutrients_list
        return ingredients

    def _extract_cooking_methods(self, recipe_text: str) -> list:
        methods = []
        for method in self.cooking_methods:
            if method in recipe_text:
                methods.append(method)
        return methods

    def get_current_solar_term(self) -> str:
        today = date.today()
        for term, dates in self.solar_terms.items():
            start_month, start_day, end_month, end_day = dates
            start_date = date(today.year, start_month, start_day)
            if start_month > end_month: # Handles December-January case
                end_date = date(today.year + 1, end_month, end_day)
            else:
                end_date = date(today.year, end_month, end_day)
            if start_date <= today <= end_date:
                return term
        return 'その他'

    def load_and_process_data(self):
        if self.df is None:
            print("エラー: DataFrameが読み込まれていません。")
            return
        self._preprocess_ingredients()
        self._preprocess_tastes()
        self._analyze_data()
        print("データ処理と分析が完了しました。")

    def get_rules(self) -> dict:
        return self.rules

    def get_nutrients_for_ingredient(self, ingredient_name: str) -> str:
        return self.ingredient_nutrients_map.get(ingredient_name, '')

    def _preprocess_ingredients(self):
        def extract(text):
            if not isinstance(text, str): return {}
            ingredients = {}
            matches = re.findall(r'【(\S+):\s*.*?】<(.*?)>', text)
            for ingredient, nutrients in matches:
                normalized_ingredient = normalize_ingredient_name(ingredient)
                ingredients[normalized_ingredient] = [n.strip() for n in nutrients.split('、')]
                self.ingredient_nutrients_map[normalized_ingredient] = f"<{nutrients}>"
            return ingredients
        if '主な食材（分量目安つき）' in self.df.columns:
            self.df['parsed_ingredients'] = self.df['主な食材（分量目安つき）'].apply(extract)
            self.df['main_ingredient'] = self.df['parsed_ingredients'].apply(self._get_main_ingredient)
        else:
            print("警告: '主な食材（分量目安つき）'列が見つかりません。")

    def _get_main_ingredient(self, ingredients: dict) -> str:
        if not isinstance(ingredients, dict):
            return None
        for ingredient_name in ingredients.keys():
            normalized_ingredient = normalize_ingredient_name(ingredient_name)
            if normalized_ingredient in self.main_keywords:
                return normalized_ingredient
        return None

    def _preprocess_tastes(self):
        def extract(name):
            if not isinstance(name, str): return 'その他'
            if '和風' in name or '豆腐' in name or '味噌' in name or 'だし' in name: return '和食'
            if 'カレー' in name or 'シチュー' in name or 'グラタン' in name or 'ソテー' in name: return '洋食'
            if '中華' in name or '麻婆' in name or '回鍋肉' in name: return '中華'
            if 'ガパオ' in name or 'タンドリー' in name or 'トムヤム' in name: return 'エスニック'
            return 'その他'
        if '料理名' in self.df.columns:
            self.df['taste'] = self.df['料理名'].apply(extract)
        else:
            print("警告: '料理名'列が見つかりません。")

    def _analyze_data(self):
        self.rules = {}
        if 'parsed_ingredients' in self.df.columns:
            all_ingredients = [ing for ingredients in self.df['parsed_ingredients'] if isinstance(ingredients, dict) for ing in ingredients.keys()]
            ingredient_counts = Counter(all_ingredients)
            self.rules['common_ingredients'] = [ing for ing, count in ingredient_counts.items() if count > 1]
            co_occurrence = Counter()
            for ingredients_dict in self.df['parsed_ingredients']:
                if not isinstance(ingredients_dict, dict): continue
                ingredients_list = list(ingredients_dict.keys())
                for pair in combinations(ingredients_list, 2):
                    co_occurrence[tuple(sorted(pair))] += 1
            self.rules['co_occurrence'] = {frozenset(pair): count for pair, count in co_occurrence.items() if count > 1}
        if 'taste' in self.df.columns and 'parsed_ingredients' in self.df.columns:
            taste_patterns = {}
            for taste in self.df['taste'].unique():
                ingredients_by_taste = [ing for sublist in self.df[self.df['taste'] == taste]['parsed_ingredients'].apply(lambda d: list(d.keys()) if isinstance(d, dict) else []) for ing in sublist]
                taste_patterns[taste] = [ing for ing, count in Counter(ingredients_by_taste).items() if count > 1]
            self.rules['taste_patterns'] = taste_patterns

    def get_random_existing_menu(self) -> dict:
        if self.df is not None and not self.df.empty and '料理名' in self.df.columns and '主な食材（分量目安つき）' in self.df.columns and '作り方' in self.df.columns and 'カロリーの目安' in self.df.columns:
            random_row = self.df.sample(n=1).iloc[0]
            ingredients_str = random_row['主な食材（分量目安つき）']
            parsed_ingredients = DataProcessor._parse_ingredients_string(ingredients_str)

            season_info = random_row.get('テイスト', 'なし')
            cooking_methods = self._extract_cooking_methods(random_row['作り方'])

            return {
                'name': random_row['料理名'],
                'main_ingredients': list(parsed_ingredients.keys()),
                'main_ingredients_full': ingredients_str,
                'recipe': random_row['作り方'],
                'calories': random_row['カロリーの目安'],
                'season': season_info,
                'taste': random_row.get('taste', 'なし'),
                'main_ingredient': self._get_main_ingredient(parsed_ingredients),
                'cooking_methods': cooking_methods
            }
        return None

    def get_existing_menu_from_ingredients(self, available_ingredients: list, used_menu_names: list, used_main_ingredients: list) -> dict:
        matched_recipes = []
        df_filtered = self.df[~self.df['料理名'].isin(used_menu_names)]

        main_ingredient_candidates = [ing for ing in self.main_keywords if ing not in used_main_ingredients]

        for index, row in df_filtered.iterrows():
            if 'parsed_ingredients' in row and isinstance(row['parsed_ingredients'], dict):
                recipe_ingredients = list(row['parsed_ingredients'].keys())
                main_ingredient = self._get_main_ingredient(row.get('parsed_ingredients', {}))

                if main_ingredient not in main_ingredient_candidates:
                    continue

                match_count = sum(1 for ing in recipe_ingredients if ing in available_ingredients)

                if match_count > 0:
                    ingredients_str = row['主な食材（分量目安つき）']
                    parsed_ingredients = DataProcessor._parse_ingredients_string(ingredients_str)
                    season_info = row.get('テイスト', 'なし')
                    cooking_methods = self._extract_cooking_methods(row['作り方'])

                    matched_recipes.append({
                        'match_count': match_count,
                        'name': row['料理名'],
                        'main_ingredients': list(parsed_ingredients.keys()),
                        'main_ingredients_full': ingredients_str,
                        'recipe': row['作り方'],
                        'calories': row['カロリーの目安'],
                        'season': season_info,
                        'source': '既存',
                        'taste': row.get('taste', 'なし'),
                        'main_ingredient': main_ingredient,
                        'cooking_methods': cooking_methods
                    })
        if matched_recipes:
            matched_recipes.sort(key=lambda x: x['match_count'], reverse=True)
            max_matches = matched_recipes[0]['match_count']
            top_matched_recipes = [r for r in matched_recipes if r['match_count'] == max_matches]
            return random.choice(top_matched_recipes)
        else:
            return None

    def get_seasonal_menu(self, available_ingredients: list, used_menu_names: list, used_main_ingredients: list) -> dict:
        current_solar_term = self.get_current_solar_term()
        if 'テイスト' not in self.df.columns:
            print("警告: 'テイスト'列が見つかりません。既存ベースの献立を試します。")
            return self.get_existing_menu_from_ingredients(available_ingredients, used_menu_names, used_main_ingredients)

        filtered_seasonal_df = self.df[self.df['テイスト'] == current_solar_term]
        filtered_seasonal_recipes = []

        main_ingredient_candidates = [ing for ing in self.main_keywords if ing not in used_main_ingredients]

        for index, row in filtered_seasonal_df.iterrows():
            if row['料理名'] in used_menu_names:
                continue

            main_ingredient = self._get_main_ingredient(row.get('parsed_ingredients', {}))
            if main_ingredient not in main_ingredient_candidates:
                continue

            recipe_ingredients = list(row['parsed_ingredients'].keys())
            match_count = sum(1 for ing in recipe_ingredients if ing in available_ingredients)

            if match_count > 0:
                ingredients_str = row['主な食材（分量目安つき）']
                parsed_ingredients = DataProcessor._parse_ingredients_string(ingredients_str)
                season_info = row.get('テイスト', 'なし')
                cooking_methods = self._extract_cooking_methods(row['作り方'])

                filtered_seasonal_recipes.append({
                    'match_count': match_count,
                    'name': row['料理名'],
                    'main_ingredients': list(parsed_ingredients.keys()),
                    'main_ingredients_full': ingredients_str,
                    'recipe': row['作り方'],
                    'calories': row['カロリーの目安'],
                    'season': current_solar_term,
                    'source': '旬（既存）',
                    'taste': row.get('taste', 'なし'),
                    'main_ingredient': main_ingredient,
                    'cooking_methods': cooking_methods
                })

        if filtered_seasonal_recipes:
            filtered_seasonal_recipes.sort(key=lambda x: x['match_count'], reverse=True)
            return random.choice(filtered_seasonal_recipes)
        return None

    def get_herb_menu(self, available_ingredients: list, herb: str, used_menu_names: list, used_main_ingredients: list) -> dict:
        filtered_df = self.df[self.df['主な食材（分量目安つき）'].str.contains(herb, case=False, na=False)]
        filtered_herb_recipes = []

        main_ingredient_candidates = [ing for ing in self.main_keywords if ing not in used_main_ingredients]

        for index, row in filtered_df.iterrows():
            if row['料理名'] in used_menu_names:
                continue

            main_ingredient = self._get_main_ingredient(row.get('parsed_ingredients', {}))
            if main_ingredient not in main_ingredient_candidates:
                continue

            recipe_ingredients = list(row['parsed_ingredients'].keys())
            match_count = sum(1 for ing in recipe_ingredients if ing in available_ingredients)

            if match_count > 0:
                ingredients_str = row['主な食材（分量目安つき）']
                parsed_ingredients = DataProcessor._parse_ingredients_string(ingredients_str)
                season_info = row.get('テイスト', 'なし')
                cooking_methods = self._extract_cooking_methods(row['作り方'])

                filtered_herb_recipes.append({
                    'match_count': match_count,
                    'name': row['料理名'],
                    'main_ingredients': list(parsed_ingredients.keys()),
                    'main_ingredients_full': ingredients_str,
                    'recipe': row['作り方'],
                    'calories': row['カロリーの目安'],
                    'season': season_info,
                    'source': 'ハーブ（既存）',
                    'taste': 'ハーブ',
                    'main_ingredient': main_ingredient,
                    'cooking_methods': cooking_methods
                })
        if filtered_herb_recipes:
            filtered_herb_recipes.sort(key=lambda x: x['match_count'], reverse=True)
            return random.choice(filtered_herb_recipes)
        return None

    def get_specific_dish_menu(self, dish_name: str, used_menu_names: list, used_main_ingredients: list) -> dict:
        filtered_df = self.df[self.df['料理名'].str.contains(dish_name, case=False, na=False)]
        if filtered_df.empty:
            return None

        # ランダムに選択する前に、既に使われた料理名やメイン食材をフィルタリング
        valid_candidates = [row for _, row in filtered_df.iterrows()
                            if row['料理名'] not in used_menu_names and
                            self._get_main_ingredient(row.get('parsed_ingredients', {})) not in used_main_ingredients]

        if not valid_candidates:
            return None

        row = random.choice(valid_candidates)
        ingredients_str = row['主な食材（分量目安つき）']
        parsed_ingredients = DataProcessor._parse_ingredients_string(ingredients_str)
        cooking_methods = self._extract_cooking_methods(row['作り方'])
        return {
            'name': row['料理名'],
            'main_ingredients': list(parsed_ingredients.keys()),
            'main_ingredients_full': ingredients_str,
            'recipe': row['作り方'],
            'calories': row['カロリーの目安'],
            'season': row.get('テイスト', 'なし'),
            'source': '料理名指定',
            'taste': row.get('taste', 'なし'),
            'main_ingredient': self._get_main_ingredient(parsed_ingredients),
            'cooking_methods': cooking_methods
        }


class RuleBasedMenuSelector:
    def __init__(self, df, data_processor):
        self.df = df
        self.data_processor = data_processor

    def get_special_menu(self, used_menu_names: list, used_main_ingredients: list, available_ingredients: list) -> dict:
        candidates = ["グラタン", "餃子のチーズ巻き", "お好み焼き"]
        random.shuffle(candidates)
        for cand_name in candidates:
            filtered_df = self.df[self.df['料理名'].str.contains(cand_name, case=False, na=False)]
            if not filtered_df.empty:
                for index, row in filtered_df.iterrows():
                    if row['料理名'] not in used_menu_names:
                        main_ingredient = self.data_processor._get_main_ingredient(row.get('parsed_ingredients', {}))
                        if main_ingredient not in used_main_ingredients:
                            ingredients_str = row['主な食材（分量目安つき）']
                            parsed_ingredients = DataProcessor._parse_ingredients_string(ingredients_str)
                            cooking_methods = self.data_processor._extract_cooking_methods(row['作り方'])
                            return {
                                'name': row['料理名'],
                                'main_ingredients': list(parsed_ingredients.keys()),
                                'main_ingredients_full': ingredients_str,
                                'recipe': row['作り方'],
                                'calories': row['カロリーの目安'],
                                'season': row.get('テイスト', 'なし'),
                                'source': '特別ルール',
                                'taste': row.get('taste', 'なし'),
                                'main_ingredient': main_ingredient,
                                'cooking_methods': cooking_methods
                            }
        return None

    def get_soup_menu(self, used_menu_names: list, used_main_ingredients: list, available_ingredients: list) -> dict:
        candidates = ["お汁", "スープ", "シチュー"]
        filtered_df = self.df[self.df['料理名'].str.contains('|'.join(candidates), case=False, na=False)]
        filtered_recipes = [row for index, row in filtered_df.iterrows() if row['料理名'] not in used_menu_names]
        if filtered_recipes:
            random.shuffle(filtered_recipes)
            for row in filtered_recipes:
                main_ingredient = self.data_processor._get_main_ingredient(row.get('parsed_ingredients', {}))
                if main_ingredient not in used_main_ingredients:
                    ingredients_str = row['主な食材（分量目安つき）']
                    parsed_ingredients = DataProcessor._parse_ingredients_string(ingredients_str)
                    cooking_methods = self.data_processor._extract_cooking_methods(row['作り方'])
                    return {
                        'name': row['料理名'],
                        'main_ingredients': list(parsed_ingredients.keys()),
                        'main_ingredients_full': ingredients_str,
                        'recipe': row['作り方'],
                        'calories': row['カロリーの目安'],
                        'season': row.get('テイスト', 'なし'),
                        'source': '特別ルール',
                        'taste': row.get('taste', 'なし'),
                        'main_ingredient': main_ingredient,
                        'cooking_methods': cooking_methods
                    }
        return None

    def get_solar_term_menu(self, used_menu_names: list, used_main_ingredients: list, available_ingredients: list, term_type: str) -> dict:
        if term_type == 'cold':
            keywords = ["冷たいレシピ", "冷たいおかず", "冷麺"]
        else: # 'nabe'
            keywords = ["鍋"]

        filtered_df = self.df[self.df['料理名'].str.contains('|'.join(keywords), case=False, na=False)]
        filtered_recipes = [row for index, row in filtered_df.iterrows() if row['料理名'] not in used_menu_names]

        if filtered_recipes:
            random.shuffle(filtered_recipes)
            for row in filtered_recipes:
                main_ingredient = self.data_processor._get_main_ingredient(row.get('parsed_ingredients', {}))
                if main_ingredient not in used_main_ingredients:
                    ingredients_str = row['主な食材（分量目安つき）']
                    parsed_ingredients = DataProcessor._parse_ingredients_string(ingredients_str)
                    cooking_methods = self.data_processor._extract_cooking_methods(row['作り方'])
                    return {
                        'name': row['料理名'],
                        'main_ingredients': list(parsed_ingredients.keys()),
                        'main_ingredients_full': ingredients_str,
                        'recipe': row['作り方'],
                        'calories': row['カロリーの目安'],
                        'season': row.get('テイスト', 'なし'),
                        'source': '特別ルール',
                        'taste': row.get('taste', 'なし'),
                        'main_ingredient': main_ingredient,
                        'cooking_methods': cooking_methods
                    }
        return None

class MenuGenerator:
    def __init__(self, df, data_processor):
        self.df = df
        self.data_processor = data_processor

    def generate_menu_from_pantry(self, available_ingredients: list, used_menu_names: list, user_ingredients: list, used_ingredients_counter: Counter, initial_pantry_list: list, used_main_ingredients: list, used_cooking_methods: set) -> dict:
        menu_candidates = []
        df_filtered = self.df[~self.df['料理名'].isin(used_menu_names)]

        def evaluate_candidate_v2(row, available_ingredients, user_ingredients, used_ingredients_counter, initial_pantry_list, used_main_ingredients, used_cooking_methods):
            if 'parsed_ingredients' not in row or not isinstance(row['parsed_ingredients'], dict):
                return -1, []

            required_ingredients = list(row['parsed_ingredients'].keys())
            matched_ingredients = [ing for ing in required_ingredients if ing in available_ingredients]
            missing_ingredients = [ing for ing in required_ingredients if ing not in available_ingredients]

            cooking_methods = self.data_processor._extract_cooking_methods(row['作り方'])

            main_ingredient = self.data_processor._get_main_ingredient(row.get('parsed_ingredients', {}))
            if main_ingredient in used_main_ingredients:
                return 0, []

            if len(matched_ingredients) == 0:
                return 0, []

            match_score = len(matched_ingredients) * 10
            missing_penalty = len(missing_ingredients) * -5

            user_ingredient_bonus = sum(1 for ing in matched_ingredients if ing in user_ingredients and used_ingredients_counter[ing] == 0) * 50
            pantry_unused_bonus = sum(1 / (1 + used_ingredients_counter[ing]) for ing in matched_ingredients if ing in initial_pantry_list) * 25
            diversity_score = sum(1 / (1 + used_ingredients_counter[ing]) for ing in matched_ingredients) * 15

            cooking_method_penalty = 0
            for method in cooking_methods:
                if method in used_cooking_methods:
                    cooking_method_penalty -= 20

            score = match_score + missing_penalty + user_ingredient_bonus + pantry_unused_bonus + diversity_score + cooking_method_penalty

            return max(0, score), missing_ingredients

        for index, row in df_filtered.iterrows():
            score, missing_ingredients = evaluate_candidate_v2(row, available_ingredients, user_ingredients, used_ingredients_counter, initial_pantry_list, used_main_ingredients, used_cooking_methods)
            if score > 0:
                season_info = row.get('テイスト', 'なし')
                cooking_methods = self.data_processor._extract_cooking_methods(row['作り方'])

                menu_candidates.append({
                    'score': score,
                    'name': row['料理名'],
                    'main_ingredients': list(row['parsed_ingredients'].keys()),
                    'main_ingredients_full': row['主な食材（分量目安つき）'],
                    'recipe': row['作り方'],
                    'calories': row['カロリーの目安'],
                    'season': season_info,
                    'source': '既存',
                    'taste': row.get('taste', 'なし'),
                    'main_ingredient': self.data_processor._get_main_ingredient(row.get('parsed_ingredients', {})),
                    'cooking_methods': cooking_methods
                })

        if not menu_candidates:
            return None

        scores = [c['score'] for c in menu_candidates]
        total_score = sum(scores)
        if total_score == 0:
            return random.choice(menu_candidates)

        weights = [s / total_score for s in scores]
        selected_menu = random.choices(menu_candidates, weights=weights, k=1)[0]
        return selected_menu

class RecipeCreator:
    def check_validity(self, recipe: str) -> bool:
        return len(recipe) >= 50 and any(method in recipe for method in ["炒める", "煮る", "焼く", "茹でる", "揚げる", "蒸す"])

class GeminiRecipeGenerator:
    def __init__(self, api_key: str, data_processor):
        if not api_key:
            raise ValueError("APIキーが設定されていません。")
        self.api_key = api_key
        try:
            genai.configure(api_key=self.api_key)
            self.model = genai.GenerativeModel('gemini-2.5-flash-lite')
        except Exception as e:
            raise ValueError(f"Gemini APIモジュールの初期化に失敗しました: {e}")
        self.data_processor = data_processor

    def _generate_json_recipe(self, prompt: str) -> dict:
        try:
            response = self.model.generate_content(prompt)
        except Exception as e:
            print(f"モデル '{self.model.model_name}' でエラーが発生しました: {e}")
            print("代替モデルに切り替えて再試行します。")
            try:
                self.model = genai.GenerativeModel('gemini-1.5-flash-latest')
                response = self.model.generate_content(prompt)
            except Exception as e:
                print(f"代替モデルでもエラーが発生しました: {e}")
                raise

        if not response or not response.text or not isinstance(response.text, str):
            raise ValueError("モデルからの応答が空または不正な形式です。")
        json_text = response.text.strip()
        if json_text.startswith('```json'):
            json_text = json_text[7:]
        if json_text.endswith('```'):
            json_text = json_text[:-3]
        json_text = json_text.strip()
        try:
            return json.loads(json_text)
        except json.JSONDecodeError as e:
            print(f"JSONパースエラーが発生しました。モデルの応答を確認してください: {response.text}")
            raise e
        except Exception as e:
            print(f"AIによる献立生成に失敗しました: {e}")
            raise

    def generate_menu_from_ingredients(self, available_ingredients: list, used_ingredients: Counter, selected_tastes: list, nutrition_priority: bool, forced_main_ingredient: str = None, used_cooking_methods: set = set()) -> dict:
        unused_ingredients = [ing for ing, count in used_ingredients.items() if count == 0 and ing in available_ingredients]
        used_ingredients_list = [ing for ing, count in used_ingredients.items() if count > 0 and ing in available_ingredients]

        main_ingredient_prompt = f"この料理の主役は**{forced_main_ingredient}**です。必ずこの食材を使ってください。" if forced_main_ingredient else ""

        taste_prompt = f"テイストは{', '.join(selected_tastes)}の中から選んでください。" if selected_tastes else ""
        nutrition_prompt = "栄養バランス（ビタミン、ミネラル、タンパク質、脂質、炭水化物、食物繊維など）が偏らないように食材を組み合わせてください。" if nutrition_priority else ""
        cooking_method_prompt = ""
        if used_cooking_methods:
            cooking_method_prompt = f"既に使われた調理法は'{', '.join(used_cooking_methods)}'です。これら以外の調理法を優先的に提案してください。"

        prompt = f"""以下の食材を主役にした料理の献立とレシピを考えてください。
レシピに使用する食材は、提供された食材リストの中から選んでください。
特に、以下の**「まだ使っていない食材」を優先して**使用し、バリエーションのある献立を作成してください。

**まだ使っていない食材:** {', '.join(unused_ingredients) if unused_ingredients else 'なし'}
**これまでに使った食材:** {', '.join(used_ingredients_list) if used_ingredients_list else 'なし'}

{taste_prompt}
{nutrition_prompt}
{main_ingredient_prompt}
{cooking_method_prompt}

- 既存の献立や、一般的な日本の家庭料理の枠にとらわれない、新しい組み合わせや調理法を考案してください。
- 各食材が最も魅力的に引き立つような、意外性のある使い方を盛り込んでください。
- 季節感や栄養バランスも考慮してください。
- 提案する料理は、既存のデータベースに存在しないユニークなものである必要があります。

分量は**1人分**で表記してください。食材には、その食材に豊富に含まれる栄養素を**最大3つ**、正確に表示してください。**<>内に栄養価**を表記してください。
料理全体のカロリーを、使用する食材の一般的なカロリーから**正確に計算して表示**してください。
主な食材のフォーマットは、`【食材名:分量】<栄養素>  `のように、各項目間にスペースを2つ入れてください。最後の項目にはカンマをつけないでください。
作り方は、**1. **から始まる番号付きの短い文章で、**最大5ステップ**で簡潔に記述してください。余分な説明や見出しは一切含めないでください。**改行は使わないでください。**
出力は、以下のJSON形式の文字列のみです。余分な文字（例：```json）や説明は一切含めないでください。
{{
  "料理名": "ここに料理名を書いてください",
  "主な食材（分量目安つき）": "ここに食材と分量、栄養価をCSVの形式（例：【キャベツ:1/4個】<食物繊維、ビタミンC>  【豚肉:200g】<タンパク質、脂質>）で書いてください",
  "カロリーの目安": "ここに計算したカロリーを〇〇〇kcalの形式で書いてください",
  "作り方": "ここにレシピを1から始まる番号付きの最大5ステップで簡潔に記述してください"
}}
食材リスト: {', '.join(available_ingredients)}"""
        generated_json = self._generate_json_recipe(prompt)
        if '作り方' in generated_json:
            generated_json['作り方'] = generated_json['作り方'].replace('\n', ' ').replace('\\n', ' ').strip()

        parsed_ingredients = self.data_processor._parse_ingredients_string(generated_json.get('主な食材（分量目安つき）', ''))
        cooking_methods = self.data_processor._extract_cooking_methods(generated_json.get('作り方', ''))

        return {
            'name': generated_json.get('料理名', "食材活用献立"),
            'main_ingredients': list(parsed_ingredients.keys()),
            'main_ingredients_full': generated_json.get('主な食材（分量目安つき）', f"{', '.join(available_ingredients)}"),
            'calories': generated_json.get('カロリーの目安', '不明'),
            'recipe': generated_json.get('作り方', 'レシピが見つかりませんでした。'),
            'season': 'なし',
            'taste': selected_tastes[0] if selected_tastes else 'その他',
            'main_ingredient': self.data_processor._get_main_ingredient(parsed_ingredients),
            'cooking_methods': cooking_methods,
            'source': 'AI生成'
        }

    def generate_specific_dish_menu(self, dish_keyword: str, available_ingredients: list, used_ingredients: Counter, used_cooking_methods: set = set(), spice_prompt_addition: str = "") -> dict:
        unused_ingredients = [ing for ing, count in used_ingredients.items() if count == 0 and ing in available_ingredients]
        used_ingredients_list = [ing for ing, count in used_ingredients.items() if count > 0 and ing in available_ingredients]

        cooking_method_prompt = ""
        if used_cooking_methods:
            cooking_method_prompt = f"既に使われた調理法は'{', '.join(used_cooking_methods)}'です。これら以外の調理法を優先的に提案してください。"

        prompt = f"""以下の食材リストの中から適切なものを選び、**「{dish_keyword}」**の献立とレシピを考えてください。
特に、以下の**「まだ使っていない食材」を優先して**使用し、レシピに組み込んでください。

**まだ使っていない食材:** {', '.join(unused_ingredients) if unused_ingredients else 'なし'}
**これまでに使った食材:** {', '.join(used_ingredients_list) if used_ingredients_list else 'なし'}
{cooking_method_prompt}
{spice_prompt_addition}

- 既存の献立や、一般的な日本の家庭料理の枠にとらわれない、新しい組み合わせや調理法を考案してください。
- 各食材が最も魅力的に引き立つような、意外性のある使い方を盛り込んでください。
- 季節感や栄養バランスも考慮してください。
- 提案する料理は、既存のデータベースに存在しないユニークなものである必要があります。

レシピに使用する食材は、提供された食材リストの中から選んでください。
分量は**1人分**で表記してください。食材には、その食材に豊富に含まれる栄養素を**最大3つ**、正確に表示してください。**<>内に栄養価**を表記してください。
料理全体のカロリーを、使用する食材の一般的なカロリーから**正確に計算して表示**してください。
主な食材のフォーマットは、`【食材名:分量】<栄養素>  `のように、各項目間にスペースを2つ入れてください。最後の項目にはカンマをつけないでください。
作り方は、**1. **から始まる番号付きの短い文章で、**最大5ステップ**で簡潔に記述してください。余分な説明や見出し（例: ポイント、材料、作り方など）は一切含めないでください。**改行は使わないでください。**
また、この料理がどのテイスト（和食、洋食、中華、エスニック、その他）に該当するかを判断し、`テイスト`にその値を記述してください。

出力は、以下のJSON形式の文字列のみです。余分な文字（例：```json）や説明は一切含めないでください。
{{
  "料理名": "ここに料理名を書いてください",
  "主な食材（分量目安つき）": "ここに食材と分量、栄養価をCSVの形式（例：【キャベツ:1/4個】<食物繊維、ビタミンC>  【豚肉:200g>）で書いてください",
  "カロリーの目安": "ここに計算したカロリーを〇〇〇kcalの形式で書いてください",
  "作り方": "ここにレシピを1から始まる番号付きの最大5ステップで簡潔に記述してください",
  "テイスト": "ここにテイスト（和食、洋食、中華、エスニック、その他）を記述してください"
}}
食材リスト: {', '.join(available_ingredients)}"""

        generated_json = self._generate_json_recipe(prompt)

        if '作り方' in generated_json:
            generated_json['作り方'] = generated_json['作り方'].replace('\n', ' ').replace('\\n', ' ').strip()

        parsed_ingredients = self.data_processor._parse_ingredients_string(generated_json.get('主な食材（分量目安つき）', ''))
        cooking_methods = self.data_processor._extract_cooking_methods(generated_json.get('作り方', ''))

        return {
            'name': generated_json.get('料理名', dish_keyword),
            'main_ingredients': list(parsed_ingredients.keys()),
            'main_ingredients_full': generated_json.get('主な食材（分量目安つき）', ''),
            'calories': generated_json.get('カロリーの目安', '不明'),
            'recipe': generated_json.get('作り方', 'レシピが見つかりませんでした。'),
            'season': 'なし',
            'taste': generated_json.get('テイスト', 'その他'),
            'main_ingredient': self.data_processor._get_main_ingredient(parsed_ingredients),
            'cooking_methods': cooking_methods,
            'source': 'AI生成:特別ルール'
        }

    def generate_herb_menu(self, ingredients: list, herb: str, used_ingredients: Counter, forced_main_ingredient: str = None, used_cooking_methods: set = set()) -> dict:
        unused_ingredients = [ing for ing, count in used_ingredients.items() if count == 0 and ing in ingredients]
        used_ingredients_list = [ing for ing, count in used_ingredients.items() if count > 0 and ing in ingredients]

        main_ingredient_prompt = f"この料理の主役は**{forced_main_ingredient}**です。必ずこの食材を使ってください。" if forced_main_ingredient else ""
        cooking_method_prompt = ""
        if used_cooking_methods:
            cooking_method_prompt = f"既に使われた調理法は'{', '.join(used_cooking_methods)}'です。これら以外の調理法を優先的に提案してください。"

        prompt = f"""以下の食材を使って、ハーブ「{herb}」を主役にした料理の献立とレシピを考えてください。
レシピに使用する食材は、提供された食材リストの中から選んでください。
特に、以下の**「まだ使っていない食材」を優先して**使用し、バリエーションのある献立を作成してください。

**まだ使っていない食材:** {', '.join(unused_ingredients) if unused_ingredients else 'なし'}
**これまでに使った食材:** {', '.join(used_ingredients_list) if used_ingredients_list else 'なし'}
{main_ingredient_prompt}
{cooking_method_prompt}

- 既存の献立や、一般的な日本の家庭料理の枠にとらわれない、新しい組み合わせや調理法を考案してください。
- 各食材が最も魅力的に引き立つような、意外性のある使い方を盛り込んでください。
- 季節感や栄養バランスも考慮してください。
- 提案する料理は、既存のデータベースに存在しないユニークなものである必要があります。

分量は**1人分**で表記してください。食材には、その食材に豊富に含まれる栄養素を**最大3つ**、正確に表示してください。**<>内に栄養価**を表記してください。
料理全体のカロリーを、使用する食材の一般的なカロリーから**正確に計算して表示**してください。
主な食材のフォーマットは、`【食材名:分量】<栄養素>  `のように、各項目間にスペースを2つ入れてください。最後の項目にはカンマをつけないでください。
作り方は、**1. **から始まる番号付きの短い文章で、**最大5ステップ**で簡潔に記述してください。余分な説明や見出し（例: ポイント、材料、作り方など）は一切含めないでください。**改行は使わないでください。**
出力は、以下のJSON形式の文字列のみです。余分な文字（例：```json）や説明は一切含めないでください。
{{
  "料理名": "ここに料理名を書いてください",
  "主な食材（分量目安つき）": "ここに食材と分量、栄養価をCSVの形式（例：【キャベツ:1/4個】<食物繊維、ビタミンC>  【豚肉:200g>）で書いてください",
  "カロリーの目安": "ここに計算したカロリーを〇〇〇kcalの形式で書いてください",
  "作り方": "ここにレシピを1から始まる番号付きの最大5ステップで簡潔に記述してください"
}}
食材リスト: {', '.join(ingredients)}"""
        generated_json = self._generate_json_recipe(prompt)
        if '作り方' in generated_json:
            generated_json['作り方'] = generated_json['作り方'].replace('\n', ' ').replace('\\n', ' ').strip()

        parsed_ingredients = self.data_processor._parse_ingredients_string(generated_json.get('主な食材（分量目安つき）', ''))
        cooking_methods = self.data_processor._extract_cooking_methods(generated_json.get('作り方', ''))

        return {
            'name': generated_json.get('料理名', f"{herb}を使った料理"),
            'main_ingredients': list(parsed_ingredients.keys()),
            'main_ingredients_full': generated_json.get('主な食材（分量目安つき）', f"{', '.join(ingredients)}, {herb}"),
            'calories': generated_json.get('カロリーの目安', '不明'),
            'recipe': generated_json.get('作り方', 'レシピが見つかりませんでした。'),
            'season': 'なし',
            'taste': 'ハーブ',
            'main_ingredient': self.data_processor._get_main_ingredient(parsed_ingredients),
            'cooking_methods': cooking_methods,
            'source': 'AI生成'
        }

    def generate_seasonal_menu(self, ingredients: list, season_name: str, used_ingredients: Counter, forced_main_ingredient: str = None, used_cooking_methods: set = set()) -> dict:
        unused_ingredients = [ing for ing, count in used_ingredients.items() if count == 0 and ing in ingredients]
        used_ingredients_list = [ing for ing, count in used_ingredients.items() if count > 0 and ing in ingredients]

        main_ingredient_prompt = f"この料理の主役は**{forced_main_ingredient}**です。必ずこの食材を使ってください。" if forced_main_ingredient else ""
        cooking_method_prompt = ""
        if used_cooking_methods:
            cooking_method_prompt = f"既に使われた調理法は'{', '.join(used_cooking_methods)}'です。これら以外の調理法を優先的に提案してください。"

        prompt = f"""以下の食材を使って、旬の食材を取り入れた献立とレシピを考えてください。
季節は**{season_name}**です。この時期に旬となる食材を優先的に使用し、栄養バランスの良い献立を作成してください。
レシピに使用する食材は、提供された食材リストの中から選んでください。
特に、以下の**「まだ使っていない食材」を優先して**使用し、バリエーションのある献立を作成してください。

**まだ使っていない食材:** {', '.join(unused_ingredients) if unused_ingredients else 'なし'}
**これまでに使った食材:** {', '.join(used_ingredients_list) if used_ingredients_list else 'なし'}
{main_ingredient_prompt}
{cooking_method_prompt}

- 既存の献立や、一般的な日本の家庭料理の枠にとらわれない、新しい組み合わせや調理法を考案してください。
- 各食材が最も魅力的に引き立つような、意外性のある使い方を盛り込んでください。
- 季節感や栄養バランスも考慮してください。
- 提案する料理は、既存のデータベースに存在しないユニークなものである必要があります。

分量は**1人分**で表記してください。食材には、その食材に豊富に含まれる栄養素を**最大3つ**、正確に表示してください。**<>内に栄養価**を表記してください。
料理全体のカロリーを、使用する食材の一般的なカロリーから**正確に計算して表示**してください。
主な食材のフォーマットは、`【食材名:分量】<栄養素>  `のように、各項目間にスペースを2つ入れてください。最後の項目にはカンマをつけないでください。
作り方は、**1. **から始まる番号付きの短い文章で、**最大5ステップ**で簡潔に記述してください。余分な説明や見出し（例: ポイント、材料、作り方など）は一切含めないでください。**改行は使わないでください。**
出力は、以下のJSON形式の文字列のみです。余分な文字（例：```json）や説明は一切含めないでください。
{{
  "料理名": "ここに料理名を書いてください",
  "主な食材（分量目安つき）": "ここに食材と分量、栄養価をCSVの形式（例：【キャベツ:1/4個】<食物繊維、ビタミンC>  【豚肉:200g>）で書いてください",
  "カロリーの目安": "ここに計算したカロリーを〇〇〇kcalの形式で書いてください",
  "作り方": "ここにレシピを1から始まる番号付きの最大5ステップで簡潔に記述してください"
}}
食材リスト: {', '.join(ingredients)}"""
        generated_json = self._generate_json_recipe(prompt)
        if '作り方' in generated_json:
            generated_json['作り方'] = generated_json['作り方'].replace('\n', ' ').replace('\\n', ' ').strip()

        parsed_ingredients = self.data_processor._parse_ingredients_string(generated_json.get('主な食材（分量目安つき）', ''))
        cooking_methods = self.data_processor._extract_cooking_methods(generated_json.get('作り方', ''))

        return {
            'name': generated_json.get('料理名', f"{season_name}の献立"),
            'main_ingredients': list(parsed_ingredients.keys()),
            'main_ingredients_full': generated_json.get('主な食材（分量目安つき）', f"{', '.join(ingredients)}"),
            'calories': generated_json.get('カロリーの目安', '不明'),
            'recipe': generated_json.get('作り方', 'レシピが見つかりませんでした。'),
            'season': season_name,
            'taste': season_name,
            'main_ingredient': self.data_processor._get_main_ingredient(parsed_ingredients),
            'cooking_methods': cooking_methods,
            'source': 'AI生成'
        }

    def rewrite_recipe(self, menu_data: dict, used_cooking_methods: set = set()) -> str:
        cooking_method_prompt = ""
        if used_cooking_methods:
            cooking_method_prompt = f"既に使われた調理法は'{', '.join(used_cooking_methods)}'です。これら以外の調理法を優先的に提案してください。"

        prompt = f"以下の献立のレシピを、1. から始まる番号付きの短い文章で、最大5ステップで簡潔に書き直してください。余分な説明や見出し（例: ポイント、材料、作り方など）は一切含めないでください。\n献立名: {menu_data['name']}\n主な食材: {menu_data['main_ingredients_full']}\n不自然なレシピ: {menu_data['recipe']}\n{cooking_method_prompt}"
        response = self.model.generate_content(prompt)
        cleaned_recipe = re.sub(r'\n+', ' ', response.text.strip()).strip()
        return cleaned_recipe

def format_ingredients_with_markers(ingredients_full_str: str, available_ingredients: list) -> str:
    if not isinstance(ingredients_full_str, str):
        return "情報なし"

    formatted_parts = []
    # `、` or `  `で分割
    parts = re.split(r'、|  ', ingredients_full_str)
    parts = [p.strip() for p in parts if p.strip()]

    # 正規化されたavailable_ingredientsリストを作成
    normalized_available_ingredients = {normalize_ingredient_name(ing) for ing in available_ingredients}

    for part in parts:
        ingredient_name_match = re.match(r'【(\S+):', part)
        if ingredient_name_match:
            ingredient_name = ingredient_name_match.group(1).strip()
            # 正規化された食材名が利用可能食材リストにないかチェック
            if normalize_ingredient_name(ingredient_name) not in normalized_available_ingredients:
                # 正規表現で【】で囲まれた部分全体をキャプチャ
                # 【食材名:分量】<栄養素>  のような形式に対応
                formatted_part = re.sub(r'【(\S+):', f'【{ingredient_name}※:', part, 1)
                formatted_parts.append(formatted_part)
            else:
                formatted_parts.append(part)
        else:
            formatted_parts.append(part)
    return "  ".join(formatted_parts)


def save_to_csv_with_deduplication(df_to_save: pd.DataFrame, file_path: str):
    column_order = ['ID', '計画(0)、作った(1)', 'テイスト', '料理名', '主な食材（分量目安つき）', 'カロリーの目安', '作り方']
    key_columns = ['料理名', '主な食材（分量目安つき）', '作り方']
    try:
        if os.path.exists(file_path):
            existing_df = pd.read_csv(file_path, encoding='utf-8-sig')
            print(f"既存のCSVファイル '{file_path}' を読み込みました。")
            existing_df.drop_duplicates(subset=key_columns, keep='first', inplace=True)
            existing_df = existing_df.loc[:, existing_df.columns.isin(column_order)]
            existing_df = existing_df.reindex(columns=column_order)
        else:
            existing_df = pd.DataFrame(columns=column_order)
            print("既存ファイルが見つからないため、新規作成します。")
    except Exception as e:
        print(f"既存のCSVの読み込み中にエラーが発生しました: {e}")
        print("新規ファイルとして保存します。")
        existing_df = pd.DataFrame(columns=column_order)
    df_to_save = df_to_save.reindex(columns=column_order)
    if '主な食材（分量目安つき）' in df_to_save.columns:
        df_to_save['主な食材（分量目安つき）'] = df_to_save['主な食材（分量目安つき）'].str.replace(r'※:', ':', regex=True).str.replace(r',', '  ', regex=False)
    if '作り方' in df_to_save.columns:
        df_to_save['作り方'] = df_to_save['作り方'].str.replace('\n', ' ', regex=False).str.replace(r'\\n', ' ', regex=False).str.strip()
    combined_df = pd.concat([existing_df, df_to_save], ignore_index=True)
    combined_df.drop_duplicates(subset=key_columns, keep='first', inplace=True)
    combined_df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\n重複を排除した新しい献立を '{file_path}' に追加保存しました。")

def is_first_or_last_week_of_month(target_date: date) -> bool:
    first_day_of_month = date(target_date.year, target_date.month, 1)
    last_day_of_month = date(target_date.year, target_date.month + 1, 1) - pd.Timedelta(days=1)
    first_week_of_month_end = first_day_of_month + pd.Timedelta(days=6 - first_day_of_month.weekday())
    last_week_of_month_start = last_day_of_month - pd.Timedelta(days=last_day_of_month.weekday())
    return target_date <= first_week_of_month_end or target_date >= last_week_of_month_start

def apply_date_based_rules(menu_plan: list, data_processor: DataProcessor, selector: RuleBasedMenuSelector) -> list:
    today = date.today()
    updated_menu_plan = menu_plan[:]
    num_days = len(updated_menu_plan)

    # 既存の3,13,23日ルール
    if today.day in [3, 13, 23]:
        updated_menu_plan.insert(0, "AI生成:特別ルール:グラタン,餃子のチーズ巻き,お好み焼き")

    # 週ルール → 1,10,30日の日付ルールに置き換え
    if today.day in [1, 10, 30]:
        if "AI生成:特別ルール" not in updated_menu_plan[0]:
            updated_menu_plan.insert(0, "AI生成:特別ルール:お汁,スープ,シチュー")

    # 新規追加：2,20,22日の麺類ルール
    if today.day in [2, 20, 22]:
        if "AI生成:特別ルール" not in updated_menu_plan[0]:
            updated_menu_plan.insert(0, "AI生成:特別ルール:パスタ,うどん,そば,ラーメン")

    updated_menu_plan = updated_menu_plan[:num_days]

    # 以下、既存の二十四節気ルールはそのまま残す
    current_solar_term = data_processor.get_current_solar_term()
    cold_terms = ["小暑", "大暑", "立秋"]
    nabe_terms = ["大雪", "冬至", "小寒", "大寒"]

    if current_solar_term in cold_terms:
        if "AI生成:特別ルール" not in updated_menu_plan[0]:
            updated_menu_plan.insert(0, "AI生成:特別ルール:冷たいレシピ,冷たいおかず,冷麺")
    elif current_solar_term in nabe_terms:
        if "AI生成:特別ルール" not in updated_menu_plan[0]:
            updated_menu_plan.insert(0, "AI生成:特別ルール:鍋")
    updated_menu_plan = updated_menu_plan[:num_days]

    return updated_menu_plan



def main():
    print("--- Google Driveをマウントします ---")
    if not mount_drive():
        print("Google Driveのマウントに失敗したため、プログラムを終了します。")
        sys.exit()

    main_keywords = load_main_keywords()

    file_name = 'kondate5_dataset.csv'
    db_file_path = ''
    print("Google Drive内で指定されたファイルを探しています...")

    search_paths = ['/content/drive/MyDrive/cook', '/content/drive/MyDrive']
    for search_root in search_paths:
        if os.path.exists(search_root):
            for root, dirs, files in os.walk(search_root):
                if file_name in files:
                    db_file_path = os.path.join(root, file_name)
                    print(f"ファイルが見つかりました: {db_file_path}")
                    break
        if db_file_path:
            break

    if not db_file_path:
        print(f"エラー: Google Drive上で '{file_name}' が見つかりません。")
        print("ファイルがマイドライブまたは共有アイテム、共有ドライブのいずれかにあるか確認してください。")
        print("また、ファイル名（大文字・小文字、拡張子を含む）が正確か確認してください。")
        sys.exit()

    df_from_drive = load_csv_from_drive(db_file_path)
    if df_from_drive is None:
        print("献立データベースの読み込みに失敗したため、プログラムを終了します。")
        sys.exit()

    data_processor = DataProcessor(df_from_drive, main_keywords)
    data_processor.load_and_process_data()
    rules = data_processor.get_rules()
    if not rules:
        print("エラー: 献立データの分析に失敗しました。データセットの形式を確認してください。")
        sys.exit()

    menu_generator = MenuGenerator(df_from_drive, data_processor)
    recipe_creator = RecipeCreator()
    rule_selector = RuleBasedMenuSelector(df_from_drive, data_processor)

    gemini_generator = None
    if api_key:
        try:
            gemini_generator = GeminiRecipeGenerator(api_key, data_processor)
            print("Gemini APIモジュールを初期化しました。")
        except ValueError as e:
            print(f"Gemini APIモジュールの初期化に失敗しました: {e}")
    else:
        print("APIキーが設定されていないため、Gemini APIは利用できません。")

    print("--- 献立生成システム ---")
    pantry_items = ["ニンジン", "タマネギ", "卵", "牛乳", "バター",
                    "塩","砂糖","酒","味噌","こしょう",
                    "みりん","小麦粉","油","植物油"]
    pantry_tracker = PantryTracker(pantry_items)
    print(f"現在の常備食材：{', '.join(pantry_items)}\n")

    main_ingredients_list = []
    print("===========================================================")
    print("  買ってきた「メイン」の食材を追加してください。")
    print("  複数の食材は「、」で区切って入力できます。")
    print("  入力を終えるには、'完了'と入力しEnterキーを押してください。")
    print("===========================================================")

    empty_lines_count = 0
    while True:
        try:
            ingredients_input = input("入力 >> ")

            if ingredients_input == "完了": # Changed condition
                break
            else:
                empty_lines_count = 0
                new_ingredients = [ing.strip() for ing in ingredients_input.split('、')]
                new_ingredients = [ing.strip() for ing in new_ingredients if ing.strip()]
                if new_ingredients:
                    main_ingredients_list.extend(new_ingredients)
                    pantry_tracker.add_ingredients(new_ingredients, is_main=True)
                    print(f"追加されたメインの食材: {', '.join(new_ingredients)}")
                    print(f"現在のメインの食材リスト: {', '.join(main_ingredients_list)}\n")
                else:
                    print("有効な食材が入力されませんでした。もう一度入力してください。\n")
        except Exception as e:
            print(f"入力中にエラーが発生しました: {e}")
            break

    other_ingredients_list = []
    print("===========================================================")
    print("  買ってきた「その他」の食材を追加してください。")
    print("  複数の食材は「、」で区切って入力できます。")
    print("  入力を終えるには、’完了'と入力し、Enterキーを押してください。")
    print("===========================================================")

    empty_lines_count = 0
    while True:
        try:
            ingredients_input = input("入力 >> ")
            if ingredients_input == "完了": # Changed condition
                break
            else:
                empty_lines_count = 0
                new_ingredients = [ing.strip() for ing in ingredients_input.split('、')]
                new_ingredients = [ing.strip() for ing in new_ingredients if ing.strip()]
                if new_ingredients:
                    other_ingredients_list.extend(new_ingredients)
                    pantry_tracker.add_ingredients(new_ingredients)
                    print(f"追加されたその他の食材: {', '.join(new_ingredients)}")
                    print(f"現在のその他の食材リスト: {', '.join(other_ingredients_list)}\n")
                else:
                    print("有効な食材が入力されませんでした。もう一度入力してください。\n")
        except Exception as e:
                print(f"入力中にエラーが発生しました: {e}")
                break

    print(f"\n合計 {len(pantry_tracker.pantry)} 個の食材を使用して献立を作成します: {', '.join(pantry_tracker.get_available_ingredients())}")

    print("\n--- 献立生成方法の選択 ---")
    available_options = ["既存ベース", "旬", "ハーブ", "AI生成"]

    print("生成したい献立の種類と回数をコンマ区切りで指定してください。")
    print("例: 既存ベース=3回, 旬=1回, ハーブ=1回, AI生成=0回 の場合 -> 3,1,1,0")
    print("特定の料理名を指定する場合: 0,0,0,5,カレー -> 5日分のカレー献立を生成")

    choice_counts_input = input("生成回数 (コンマ区切り) > ")

    specific_dish_name = None
    menu_plan_choices = []
    original_menu_plan = []

    try:
        parts = [p.strip() for p in choice_counts_input.split(',')]
        if len(parts) >= 5 and not parts[4].isdigit():
            specific_dish_name = parts[4]
            total_count = sum(int(c) for c in parts[:4] if c.isdigit())
            if total_count == 0:
                print("有効な回数が指定されていません。デフォルトの5日分を生成します。")
                total_count = 5

            for _ in range(total_count):
                menu_plan_choices.append(f"料理名指定:{specific_dish_name}")
                original_menu_plan.append(f"料理名指定:{specific_dish_name}")
            print(f"「{specific_dish_name}」を主役にした献立を{total_count}日分生成します。")
        else:
            choice_counts = [int(count.strip()) for count in parts]
            if len(choice_counts) != len(available_options):
                print("入力された回数の数が選択肢の数と一致しません。デフォルトの5日分既存ベース献立を生成します。")
                choice_counts = [5, 0, 0, 0]

            for count, option in zip(choice_counts, available_options):
                menu_plan_choices.extend([option] * count)
                original_menu_plan.extend([option] * count)
    except (ValueError, IndexError):
        print("無効な入力です。デフォルトの5日分既存ベース献立を生成します。")
        choice_counts = [5, 0, 0, 0]
        for count, option in zip(choice_counts, available_options):
            menu_plan_choices.extend([option] * count)
            original_menu_plan.extend([option] * count)

    menu_plan_choices = apply_date_based_rules(menu_plan_choices, data_processor, rule_selector)

    # 日付ベースの特別ルールが適用された場合、元のリストにも追加
    for i, choice in enumerate(menu_plan_choices):
        if choice.startswith("AI生成:特別ルール"):
            original_menu_plan.insert(i, choice)
            original_menu_plan = original_menu_plan[:len(menu_plan_choices)]
            break

    random.shuffle(menu_plan_choices)

    weekly_menu_data = []
    used_menu_names = set()
    used_ingredients_counter = Counter()
    used_main_ingredients_this_run = set()
    used_cooking_methods = set()

    for selected_option in menu_plan_choices:
        print(f"\n--- 献立生成: {selected_option}で生成中 ---")
        menu_data = None
        final_taste = "なし"
        available_ingredients = list(pantry_tracker.get_available_ingredients())
        user_ingredients = list(pantry_tracker.get_user_ingredients())
        initial_pantry_list = list(pantry_tracker.get_initial_pantry_list())
        forced_main_ingredient = None

        source_type = '既存'
        if selected_option.startswith("AI生成"):
            source_type = 'AI生成'
            if "特別ルール" in selected_option:
                source_type = 'AI生成:特別ルール'
        elif selected_option == "旬":
            source_type = '旬'
        elif selected_option == "ハーブ":
            source_type = 'ハーブ'
        elif selected_option.startswith("料理名指定"):
            source_type = '料理名指定'

        if selected_option.startswith("料理名指定"):
            dish_keyword = selected_option.split(':')[1]

            # カレーの場合、AI生成を優先するロジックに変更
            if dish_keyword == 'カレー':
                menu_data = None # 初期化
                if gemini_generator:
                    try:
                        curry_spices = ['クミン', 'コリアンダー', 'ターメリック', 'レッドペッパー', 'ガラムマサラ', 'カルダモン', 'オールスパイス', 'クローブ']
                        available_spices = [s for s in curry_spices if normalize_ingredient_name(s) in [normalize_ingredient_name(ing) for ing in available_ingredients]]

                        spice_prompt_addition = ""
                        if available_spices:
                            spice_prompt_addition = f"カレーの風味を出すために、以下のスパイスを積極的に使ってください: {', '.join(available_spices)}。"
                        else:
                            spice_prompt_addition = "スパイスがないため、市販のカレールーを主役にした献立を提案してください。"

                        menu_data = gemini_generator.generate_specific_dish_menu(dish_keyword, available_ingredients, used_ingredients_counter, used_cooking_methods, spice_prompt_addition)
                        final_taste = menu_data.get('taste', '洋食')
                    except Exception as e:
                        print(f"AIによる献立生成に失敗しました: {e}")
                        print("既存データからのフォールバックを試みます。")

                # AI生成が失敗した場合、またはAPIキーがない場合
                if not menu_data:
                    menu_data = data_processor.get_specific_dish_menu(dish_keyword, list(used_menu_names), list(used_main_ingredients_this_run))
                    if menu_data:
                        final_taste = menu_data.get('taste', 'その他')
                    else:
                        print("AI生成も既存データからの検索も失敗しました。")

            # カレー以外の特定の料理名の場合
            else:
                menu_data = data_processor.get_specific_dish_menu(dish_keyword, list(used_menu_names), list(used_main_ingredients_this_run))

                if not menu_data:
                    print(f"指定された料理名「{dish_keyword}」の献立が既存データに見つかりませんでした。AIによる自動生成を試みます。")
                    if gemini_generator:
                        try:
                            menu_data = gemini_generator.generate_specific_dish_menu(dish_keyword, available_ingredients, used_ingredients_counter, used_cooking_methods)
                            final_taste = menu_data.get('taste', 'その他')
                        except Exception as e:
                            print(f"AIによる献立生成に失敗しました: {e}")
                    if not menu_data:
                        print("AI生成モジュールが初期化されていないか、生成に失敗しました。通常の生成を試みます。")
                        selected_option = "既存ベース"
                else:
                    final_taste = menu_data.get('taste', 'その他')

        if selected_option.startswith("AI生成:特別ルール"):
            if not gemini_generator:
                print("AI生成モジュールが初期化されていないため、特別ルールをスキップし、既存データからランダムな献立を提案します。")
                menu_data = data_processor.get_random_existing_menu()
                final_taste = menu_data.get('taste', 'なし')
            else:
                rule_keywords = selected_option.split(':')[2].split(',')
                dish_keyword = random.choice(rule_keywords)
                print(f"特別ルールにより、「{dish_keyword}」を主役にAIで献立を生成します。")

                try:
                    menu_data = gemini_generator.generate_specific_dish_menu(dish_keyword, available_ingredients, used_ingredients_counter, used_cooking_methods)
                    final_taste = menu_data.get('taste', 'その他')
                except Exception as e:
                    print(f"AIによる特別ルール献立生成に失敗しました: {e}")
                    menu_data = None


        if selected_option == "AI生成":
            print("\n--- AI生成献立オプションの選択 ---")

            print("テイストと栄養バランスの有無をコンマ区切りで指定してください。")
            print("テイスト: 1:和食, 2:洋食, 3:中華, 4:エスニック, 5:その他")
            print("栄養バランス: 1:優先する, 2:優先しない")
            print("例: 和食と栄養バランスを考慮する場合 -> 1,1")

            tastes_map = {'1': '和食', '2': '洋食', '3': '中華', '4': 'エスニック', '5': 'その他'}
            selected_tastes = []
            nutrition_priority = False

            try:
                ai_options_input = input("テイスト, 栄養バランス > ")
                choices = [c.strip() for c in ai_options_input.split(',')]

                if len(choices) >= 1:
                    taste_choice = choices[0]
                    if taste_choice in tastes_map:
                        selected_tastes.append(tastes_map[taste_choice])

                if len(choices) >= 2:
                    nutrition_choice = choices[1]
                    if nutrition_choice == '1':
                        nutrition_priority = True
            except (ValueError, KeyError):
                print("無効な入力です。全てのテイストを対象とし、栄養バランスは考慮しません。")
                selected_tastes = []
                nutrition_priority = False

            if gemini_generator:
                unused_user_mains = pantry_tracker.get_unused_user_main_ingredients()
                if unused_user_mains:
                    forced_main_ingredient = random.choice(unused_user_mains)
                    print(f"AI生成献立のメイン食材として「{forced_main_ingredient}」を確実に使用します。")

                try:
                    menu_data = gemini_generator.generate_menu_from_ingredients(available_ingredients, used_ingredients_counter, selected_tastes, nutrition_priority, forced_main_ingredient=forced_main_ingredient, used_cooking_methods=used_cooking_methods)
                    final_taste = selected_tastes[0] if selected_tastes else 'その他'
                except Exception as e:
                    print(f"AIによる献立生成に失敗しました: {e}")
                    print("既存データからランダムな献立をフォールバックとして提案します。")
                    menu_data = data_processor.get_random_existing_menu()
                    final_taste = menu_data.get('taste', 'なし')
            else:
                print("AI生成モジュールが初期化されていないため、既存データからランダムな献立を提案します。")
                menu_data = data_processor.get_random_existing_menu()
                final_taste = menu_data.get('taste', 'なし')

        elif selected_option == "旬":
            current_solar_term = data_processor.get_current_solar_term()
            menu_data = data_processor.get_seasonal_menu(available_ingredients, list(used_menu_names), list(used_main_ingredients_this_run))
            final_taste = menu_data.get('taste', 'なし') if menu_data else 'なし'

            if not menu_data and gemini_generator:
                print("旬に合う既存献立が見つかりませんでした。AI生成を試みます。")
                unused_user_mains = pantry_tracker.get_unused_user_main_ingredients()
                forced_main_ingredient = None
                if unused_user_mains and random.random() < 0.5: # 50% chance
                    forced_main_ingredient = random.choice(unused_user_mains)
                    print(f"AI生成献立のメイン食材として「{forced_main_ingredient}」を使用する可能性があります。")
                try:
                    menu_data = gemini_generator.generate_seasonal_menu(available_ingredients, current_solar_term, used_ingredients_counter, forced_main_ingredient=forced_main_ingredient, used_cooking_methods=used_cooking_methods)
                    final_taste = current_solar_term
                except Exception as e:
                    print(f"Gemini APIによる献立生成に失敗しました: {e}")
                    print("既存データからランダムな献立をフォールバックとして提案します。")
                    menu_data = data_processor.get_random_existing_menu()
            elif not menu_data:
                print("AI生成モジュールが初期化されていないか、生成に失敗しました。既存データからランダムな献立を提案します。")
                menu_data = data_processor.get_random_existing_menu()

        elif selected_option == "ハーブ":
            herb_input = input("使用したいハーブ名を入力してください > ")
            menu_data = data_processor.get_herb_menu(available_ingredients, herb_input, list(used_menu_names), list(used_main_ingredients_this_run))
            final_taste = menu_data.get('taste', 'なし') if menu_data else 'なし'

            if not menu_data and gemini_generator:
                print("指定されたハーブを含む既存献立が見つかりませんでした。AI生成を試みます。")
                unused_user_mains = pantry_tracker.get_unused_user_main_ingredients()
                forced_main_ingredient = None
                if unused_user_mains and random.random() < 0.5: # 50% chance
                    forced_main_ingredient = random.choice(unused_user_mains)
                    print(f"AI生成献立のメイン食材として「{forced_main_ingredient}」を使用する可能性があります。")
                try:
                    menu_data = gemini_generator.generate_herb_menu(available_ingredients, herb_input, used_ingredients_counter, forced_main_ingredient=forced_main_ingredient, used_cooking_methods=used_cooking_methods)
                    final_taste = "ハーブ"
                except Exception as e:
                    print(f"Gemini APIによる献立生成に失敗しました: {e}")
                    print("既存データからランダムな献立をフォールバックとして提案します。")
                    menu_data = data_processor.get_random_existing_menu()
            elif not menu_data:
                print("AI生成モジュールが初期化されていないか、生成に失敗しました。既存データからランダムな献立を提案します。")
                menu_data = data_processor.get_random_existing_menu()

        elif selected_option == "既存ベース":
            menu_data = menu_generator.generate_menu_from_pantry(available_ingredients, list(used_menu_names), user_ingredients, used_ingredients_counter, initial_pantry_list, list(used_main_ingredients_this_run), used_cooking_methods)
            final_taste = menu_data.get('taste', 'なし') if menu_data else 'なし'

            if not menu_data:
                print("既存データやルールベースで献立が見つかりませんでした。AIによる献立生成を試みます。")
                if gemini_generator:
                    unused_user_mains = pantry_tracker.get_unused_user_main_ingredients()
                    if unused_user_mains:
                        forced_main_ingredient = random.choice(unused_user_mains)
                        print(f"AI生成献立のメイン食材として「{forced_main_ingredient}」を確実に使用します。")
                    try:
                        menu_data = gemini_generator.generate_menu_from_ingredients(available_ingredients, used_ingredients_counter, [], False, forced_main_ingredient=forced_main_ingredient, used_cooking_methods=used_cooking_methods)
                        final_taste = menu_data.get('taste', 'なし')
                    except Exception as e:
                        print(f"AIによる献立生成に失敗しました: {e}")

            if not menu_data:
                print("AIによる生成も失敗しました。ランダムな献立を提案します。")
                menu_data = data_processor.get_random_existing_menu()

        if menu_data:
            if 'name' in menu_data:
                used_menu_names.add(menu_data['name'])

            ingredients_to_consume = re.findall(r'(\S+):', menu_data['main_ingredients_full'])
            for ing in ingredients_to_consume:
                normalized_ing = normalize_ingredient_name(ing)
                used_ingredients_counter[normalized_ing] += 1

            if menu_data.get('main_ingredient'):
                normalized_main = normalize_ingredient_name(menu_data['main_ingredient'])
                pantry_tracker.consume_ingredients([normalized_main])
                used_main_ingredients_this_run.add(normalized_main)

            if source_type.startswith("AI生成") and forced_main_ingredient:
                pantry_tracker.consume_ingredients([forced_main_ingredient])
                used_main_ingredients_this_run.add(forced_main_ingredient)

            simple_recipe = menu_data.get('recipe', "簡易レシピ: 主な食材を組み合わせて、お好みの調理法で調理してください。")
            if not recipe_creator.check_validity(simple_recipe) and gemini_generator:
                print("レシピに不自然な点が見つかりました。")
                try:
                    menu_data['recipe'] = gemini_generator.rewrite_recipe(menu_data, used_cooking_methods)
                except Exception as e:
                    print(f"Gemini APIによるレシピ修正に失敗しました: {e}")

            menu_data['recipe'] = menu_data['recipe'].replace('\n', ' ').replace('\\n', ' ').strip()

            if 'cooking_methods' in menu_data:
                used_cooking_methods.update(menu_data['cooking_methods'])

            formatted_ingredients = format_ingredients_with_markers(menu_data.get('main_ingredients_full', '情報なし'), available_ingredients)
            formatted_ingredients = formatted_ingredients.replace('、', '  ')

            new_id = generate_menu_id(final_taste, len(weekly_menu_data) + 1)

            weekly_menu_data.append({
                'ID': new_id,
                '計画(0)、作った(1)': 0,
                'テイスト': final_taste,
                '料理名': menu_data['name'],
                '主な食材（分量目安つき）': formatted_ingredients,
                'カロリーの目安': menu_data.get('calories', '不明'),
                '作り方': menu_data['recipe'],
                'source_type': selected_option
            })
        else:
            print("献立が見つからなかったため、次の献立に進みます。")

    # 最終的な献立表を、元の入力順に並べ替える
    if weekly_menu_data:
        # ソート順を定義
        sort_order = {
            '料理名指定': 5,
            'AI生成:特別ルール': 6,
            '旬': 2,
            'ハーブ': 3,
            '既存ベース': 1,
            'AI生成': 4
        }

        # ソートキーを作成
        def get_sort_key(item):
            for original_choice in original_menu_plan:
                # 料理名指定の場合は、部分一致で判定
                if original_choice.startswith("料理名指定"):
                    if item['source_type'].startswith("料理名指定"):
                        return original_menu_plan.index(original_choice)
                # 特別ルールも部分一致で判定
                elif original_choice.startswith("AI生成:特別ルール"):
                    if item['source_type'].startswith("AI生成:特別ルール"):
                        return original_menu_plan.index(original_choice)
                elif item['source_type'] == original_choice:
                    return original_menu_plan.index(original_choice)
            # 該当しない場合はリストの最後に追加
            return len(original_menu_plan)

        weekly_menu_data.sort(key=get_sort_key)


    print("\n" + "="*40)
    print("今週の献立表 ({}日分)".format(len(weekly_menu_data)))
    print("="*40)

    if weekly_menu_data:
        # `source_type`は表示しないため、表示用の新しいDataFrameを作成
        weekly_menu_df_display = pd.DataFrame(weekly_menu_data)
        if 'source_type' in weekly_menu_df_display.columns:
            weekly_menu_df_display = weekly_menu_df_display.drop(columns=['source_type'])

        column_order = ['ID', '計画(0)、作った(1)', 'テイスト', '料理名', '主な食材（分量目安つき）', 'カロリーの目安', '作り方']
        weekly_menu_df_display = weekly_menu_df_display.reindex(columns=column_order)
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        pd.set_option('display.max_colwidth', None)

        html_output = weekly_menu_df_display.to_html(escape=False, index=False)

        html_with_style = f"""
        <style>
            .dataframe {{ text-align: left !important; }}
            .dataframe th {{ text-align: left !important; }}
            .dataframe td {{ text-align: left !important; }}
        </style>
        {html_output}
        """
        display(HTML(html_with_style))
        save_to_csv_with_deduplication(pd.DataFrame(weekly_menu_data), db_file_path)

    else:
        print("献立を作成できませんでした。")

if __name__ == "__main__":
    main()

--- Google Driveをマウントします ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Driveをマウントしました。
メイン食材キーワードを固定値で設定しました。
Google Drive内で指定されたファイルを探しています...
ファイルが見つかりました: /content/drive/MyDrive/cook/kondate5_dataset.csv
'/content/drive/MyDrive/cook/kondate5_dataset.csv' からCSVデータをDataFrameに読み込みました。
データ処理と分析が完了しました。
Gemini APIモジュールを初期化しました。
--- 献立生成システム ---
現在の常備食材：ニンジン, タマネギ, 卵, 牛乳, バター, 塩, 砂糖, 酒, 味噌, こしょう, みりん, 小麦粉, 油, 植物油

  買ってきた「メイン」の食材を追加してください。
  複数の食材は「、」で区切って入力できます。
  入力を終えるには、'完了'と入力しEnterキーを押してください。
入力 >> 完了
  買ってきた「その他」の食材を追加してください。
  複数の食材は「、」で区切って入力できます。
  入力を終えるには、’完了'と入力し、Enterキーを押してください。
入力 >> クローブ、クミン、レッドペッパー、牛肉、豆腐
追加されたその他の食材: クローブ, クミン, レッドペッパー, 牛肉, 豆腐
現在のその他の食材リスト: クローブ, クミン, レッドペッパー, 牛肉, 豆腐

入力 >> 0,0,0,5,カレー
追加されたその他の食材: 0,0,0,5,カレー
現在のその他の食材リスト: クローブ, クミン, レッドペッパー, 牛肉, 豆腐, 0,0,0,5,カレー

入力 >> 完了

合計 20 個の食材を使用して献立を作成します: ニンジン, タマネギ, 卵, 牛乳, バター, 塩, 砂糖, 酒, 味噌, こしょう, みりん, 小麦粉, 油, 植物油, クロー

ID,計画(0)、作った(1),テイスト,料理名,主な食材（分量目安つき）,カロリーの目安,作り方
250906-E-01,0,エスニック,スパイシー牛ひき肉と豆腐のクミン風クレープ,【牛肉:100g】<タンパク質 鉄分 亜鉛> 【豆腐:100g】<タンパク質 カルシウム イソフラボン> 【ニンジン:50g】<β-カロテン ビタミンA> 【タマネギ:50g】<ケルセチン ビタミンC> 【卵:1個】<タンパク質 ビタミンD コリン> 【牛乳:50ml】<カルシウム タンパク質 ビタミンB12> 【小麦粉:50g】<炭水化物 食物繊維> 【バター:10g】<脂質 ビタミンA> 【植物油:5g】<脂質> 【クミン:小さじ1/2】<> 【レッドペッパー:少々】<> 【クローブ:2粒】<> 【塩:少々】<> 【こしょう:少々】<>,550kcal,1. 牛肉、ニンジン、タマネギをみじん切りにし、クミン、レッドペッパー、クローブ、塩、こしょうと共に炒める。2. 豆腐は崩して1に加えて炒め合わせる。3. 卵、牛乳、小麦粉、塩少々を混ぜてクレープ生地を作り、バターを熱したフライパンで薄く焼く。4. 焼きあがったクレープに2の具材を乗せて包む。5. お好みで味噌を少量溶かしたソースを添える。
250906-E-02,0,エスニック,クミン香る豆腐と卵のココナッツミルクカレースープ,【豆腐:150g】<タンパク質 カルシウム イソフラボン> 【卵:1個】<タンパク質 ビタミンD ビタミンB12> 【ココナッツミルク※:200ml】<脂質 カリウム マグネシウム> 【クミン:小さじ1】<> 【レッドペッパー:少々】<> 【クローブ:2粒】<> 【ニンジン:1/4本】<β-カロテン 食物繊維 ビタミンK> 【タマネギ:1/4個】<ケルセチン ビタミンC カリウム> 【植物油:小さじ1】<>,480kcal,1. 豆腐は水切りし、一口大に切る。ニンジン、タマネギはみじん切りにする。クローブはすり潰す。 2. 鍋に植物油、クミン、レッドペッパー、クローブを熱し、香りを出す。 3. タマネギ、ニンジンを加えてしんなりするまで炒める。 4. ココナッツミルク、豆腐を加えて煮立たせ、塩で味を調える。 5. 卵を割り入れ、卵がお好みの固さになるまで加熱する。
250906-O-03,0,その他,クミン香る豆腐と卵のミルクポタージュカレー,"【ニンジン:1/2本】<β-カロテン, ビタミンK, 食物繊維> 【タマネギ:1/4個】<ビタミンC, カリウム> 【卵:1個】<タンパク質, ビタミンD, コリン> 【牛乳:200ml】<カルシウム, タンパク質, ビタミンB12> 【バター:10g】<脂質, ビタミンA> 【豆腐:100g】<タンパク質, カルシウム, イソフラボン> 【クミン:小さじ1/2】<鉄分, 食物繊維> 【レッドペッパー:少々】<カプサイシン, ビタミンA> 【クローブ:2粒】<抗酸化物質, 鉄分> 【小麦粉:大さじ1】<炭水化物, 食物繊維> 【植物油:小さじ1】<ビタミンE> 【カレー粉※:小さじ1】<様々なスパイスの複合栄養> 【塩:少々】<ナトリウム> 【砂糖:少々】<炭水化物>",450kcal,1. ニンジンとタマネギを細かく刻み、豆腐を崩す。 2. 鍋に油とバターを熱し、ニンジンとタマネギを柔らかくなるまで煮る。 3. スパイス類と小麦粉を加えて炒め、牛乳と豆腐を加えてさらに煮込む。 4. 卵を溶きほぐして加え、半熟になったら塩と砂糖で味を調える。 5. 器に盛り付け、お好みでこしょうを振る。
250906-E-04,0,エスニック,スパイス香るクローブ香る豆腐と卵のカレー風,【豆腐:1丁(300g)】<タンパク質 カルシウム イソフラボン> 【卵:1個】<タンパク質 ビタミンD レシチン> 【ニンジン:1/2本(50g)】<β-カロテン カリウム 食物繊維> 【タマネギ:1/4個(50g)】<ケルセチン ビタミンC 食物繊維> 【牛乳:50ml】<カルシウム タンパク質 ビタミンB12> 【バター:5g】<脂質 ビタミンA> 【クローブ:2粒】<鎮静作用 抗酸化作用> 【クミン:小さじ1/2】<消化促進 鉄分> 【レッドペッパー:少々】<カプサイシン ビタミンA> 【カレー粉※:小さじ1】<様々なスパイスのブレンド 風味>,350kcal,1. 豆腐は水切りし、一口大に切る。ニンジン、タマネギはみじん切りにする。 2. フライパンにバターと植物油（分量外）を熱し、ニンジンとタマネギを弱火でじっくり炒める。 3. クローブはすり潰し、クミン、レッドペッパー、カレー粉と共に加えて香りを出す。 4. 豆腐を加え、牛乳を加えて煮立たせないように温める。 5. 溶き卵を回し入れ、半熟状になったら塩で味を調える。
250906-E-05,0,エスニック,スパイス香るクローブ風味の豆腐カレーミルク,【豆腐:150g】<タンパク質 カルシウム マグネシウム> 【ニンジン:1/4本】<β-カロテン カリウム> 【タマネギ:1/4個】<ケルセチン ビタミンC> 【牛乳:100ml】<カルシウム タンパク質 ビタミンD> 【バター:5g】<脂質 ビタミンA> 【クミン:小さじ1/2】<食物繊維 鉄> 【レッドペッパー:少々】<カプサイシン> 【クローブ:2粒】<抗酸化物質 ミネラル> 【植物油:小さじ1】<脂質> 【小麦粉:小さじ1】<炭水化物> 【塩:少々】<> 【砂糖:少々】<>,325kcal,1. 豆腐は水切りし、フォークで潰す。ニンジンとタマネギはすりおろす。2. フライパンに植物油とバターを熱し、クミン、クローブ、レッドペッパーを香りが立つまで炒める。3. 潰した豆腐、すりおろしたニンジンとタマネギ、小麦粉を加えて弱火で炒め合わせる。4. 牛乳を少しずつ加えながら混ぜ、とろみがつくまで加熱し、塩、砂糖で味を調える。


既存のCSVファイル '/content/drive/MyDrive/cook/kondate5_dataset.csv' を読み込みました。

重複を排除した新しい献立を '/content/drive/MyDrive/cook/kondate5_dataset.csv' に追加保存しました。


#自作のレシピをkondate5_dataset.csvに登録するpython